### Part 1: PHPMyAdmin

**Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do.**

```sql
SELECT 
	name as Facility
FROM 
	Facilities
WHERE
	membercost != 0;
```

**Q2: How many facilities do not charge a fee to members?** 

**A2:** 4

```sql
SELECT 
	COUNT(name) as number_of_free_facilities
FROM 
	Facilities
WHERE
	membercost = 0;
```

**Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.**

```sql
SELECT
	facid
    , name as Facility
    , membercost
    , monthlymaintenance
FROM
	Facilities
WHERE
	membercost > 0
    AND
	membercost < (.2 * (monthlymaintenance));
```

**Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator.**

```sql
SELECT
	*
FROM
	Facilities
WHERE
	facid IN (1, 5);
```

**Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than \$100. Return the name and monthly maintenance of the facilities
in question.**

```sql
SELECT
	name as Facility
    , monthlymaintenance
    , CASE
    	WHEN monthlymaintenance > 100 THEN 'expensive'
        ELSE 'cheap'
      END as maintenance_bucket
FROM
	Facilities;
```

**Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution.**

```sql
SELECT
	firstname
    , surname
FROM
	Members
WHERE
	joindate = (SELECT MAX(joindate) FROM Members);
```

**Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.**

```sql
SELECT DISTINCT
	CONCAT(b.name, ' (court): ', c.firstname, ' ', c.surname) as tennis_booking
FROM
	Bookings a
INNER JOIN
	Facilities b
ON
	a.facid = b.facid
INNER JOIN
	Members c
ON
	a.memid = c.memid
WHERE
	b.name LIKE '%tennis_court%'
ORDER BY
	c.surname
    , b.name;
```

**Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than \$30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.**

```sql
SELECT
	b.name as Facility
	, CONCAT(c.firstname, ' ', c.surname) as name
	, CASE
    	WHEN a.memid = 0 THEN a.slots * b.guestcost
        ELSE a.slots * b.membercost
    END as cost
FROM	
	Bookings a
INNER JOIN
	Facilities b
ON
	a.facid = b.facid
INNER JOIN
	Members c
ON
	a.memid = c.memid
WHERE
	(CASE
    	WHEN a.memid = 0 THEN a.slots * b.guestcost
        ELSE a.slots * b.membercost
    END) > 30
    AND 
    CAST(starttime as date) = '2012-09-14'
ORDER BY
	cost DESC;
```

**Q9: This time, produce the same result as in Q8, but using a subquery.**

```sql
SELECT
	*
FROM (
    SELECT
        b.name as Facility
        , CONCAT(c.firstname, ' ', c.surname) as name
        , CASE
            WHEN a.memid = 0 THEN a.slots * b.guestcost
            ELSE a.slots * b.membercost
        END as cost
    FROM	
        Bookings a
    INNER JOIN
        Facilities b
    ON
        a.facid = b.facid
    INNER JOIN
        Members c
    ON
        a.memid = c.memid
    WHERE
        CAST(starttime as date) = '2012-09-14') subquery
WHERE
	cost > 30
ORDER BY
	cost DESC;
```

### Part 2: SQLite

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('sqlite_db_pythonsqlite.db')

**Q10: Produce a list of facilities with a total revenue less than 1,000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!**

In [3]:
pd.read_sql_query('''
SELECT 
    *
FROM
    (
        SELECT
            b.name as Facility
            , SUM(
                CASE 
                    WHEN a.memid = 0 THEN a.slots * b.guestcost
                    ELSE a.slots * b.membercost
                END
            ) as total_revenue   
        FROM
            Bookings a
        INNER JOIN
            Facilities b
        ON
            a.facid = b.facid
        GROUP BY
            b.name
    ) subquery
WHERE
    total_revenue < 1000
ORDER BY
    total_revenue DESC;

''', con = conn)

,Facility,total_revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


**Q11: Produce a report of members and who recommended them in alphabetic surname, firstname order**

In [4]:
pd.read_sql_query('''
SELECT
    m1.surname || ' ' || m1.firstname as member
    , m2.surname || ' ' || m2.firstname as recommendedby
FROM
    Members m1
LEFT JOIN
    Members m2
ON
    m1.recommendedby = m2.memid
ORDER BY
    member
''', con = conn)

,member,recommendedby
0,Bader Florence,Stibbons Ponder
1,Baker Anne,Stibbons Ponder
2,Baker Timothy,Farrell Jemima
3,Boothe Tim,Rownam Tim
4,Butters Gerald,Smith Darren
5,Coplin Joan,Baker Timothy
6,Crumpet Erica,Smith Tracy
7,Dare Nancy,Joplette Janice
8,Farrell David,None
9,Farrell Jemima,None


**Q12: Find the facilities with their usage by member, but not guests**

In [5]:
pd.read_sql_query('''
SELECT
    f1.name as Facility
    , SUM(30 * (b1.slots)) as total_member_usage_minutes
FROM
    Bookings b1
INNER JOIN
    Facilities f1
ON
    b1.facid = f1.facid
WHERE
    b1.memid != 0
GROUP BY
    Facility
ORDER BY
    total_member_usage_minutes DESC
''', con = conn)

,Facility,total_member_usage_minutes
0,Badminton Court,32580
1,Tennis Court 1,28710
2,Massage Room 1,26520
3,Tennis Court 2,26460
4,Snooker Table,25800
5,Pool Table,25680
6,Table Tennis,23820
7,Squash Court,12540
8,Massage Room 2,1620


**Q13: Find the facilities usage by month, but not guests**

In [6]:
pd.read_sql_query('''
SELECT
    f1.name as Facility
    , strftime('%m', b1.starttime) as Month
    , SUM(30 * (b1.slots)) as total_member_usage_minutes
FROM
    Bookings b1
INNER JOIN
    Facilities f1
ON
    b1.facid = f1.facid
WHERE
    b1.memid != 0
GROUP BY
    Facility
    , Month
ORDER BY
    total_member_usage_minutes DESC
''', con = conn)

,Facility,Month,total_member_usage_minutes
0,Badminton Court,09,15210
1,Pool Table,09,13290
2,Tennis Court 1,09,12510
3,Badminton Court,08,12420
4,Tennis Court 2,09,12420
5,Snooker Table,09,12120
6,Massage Room 1,09,12060
7,Table Tennis,09,12000
8,Tennis Court 2,08,10350
9,Tennis Court 1,08,10170


In [7]:
conn.close()